In [11]:
import warnings
import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import matplotlib.pyplot as plt

In [24]:
model_name= 'Fruits.h5'
batch_size = 32
num_classes = 131
epochs = 10

path_to_train = '../input/fruits/fruits/fruits/Training'
path_to_test = '../input/fruits/fruits/fruits/Test'

In [3]:
Generator = ImageDataGenerator()
train_data = Generator.flow_from_directory(path_to_train, (100, 100), batch_size=batch_size)

test_data = Generator.flow_from_directory(path_to_test, (100, 100), batch_size=batch_size)

In [15]:
vgg = VGG16(input_shape= [100,100] + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
  layer.trainable = False
x = Flatten()(vgg.output)
prediction = Dense(num_classes, activation='softmax')(x)

In [16]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

In [17]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)

In [19]:
history = model.fit_generator(
  train_data,
  validation_data=test_data,
  epochs=epochs,
  steps_per_epoch=len(train_data) // batch_size,
  validation_steps=len(test_data) // batch_size,
)

In [21]:
# loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [22]:
# accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')

plt.legend()
plt.show()

In [23]:
test_loss, test_acc = model.evaluate(train_data, verbose=1)
print(test_acc*100)

In [26]:
model.save(model_name)